# LANGCHAIN W/ DUCKDB

[original blog from medium](https://betterprogramming.pub/analyze-canadian-open-data-inventory-with-langchain-openai-and-duckdb-f747245eaf4b)

## SETUP

In [13]:
# load api key
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

- [initialize a database in langchain](https://python.langchain.com/v0.2/docs/integrations/tools/sql_database/#initialize-database)

In [43]:
import duckdb
from sqlalchemy import create_engine, Column, Integer, Sequence, String

In [3]:
print(duckdb.__version__)

1.0.0


## CREATE DATABASE

[blog](https://betterprogramming.pub/analyze-canadian-open-data-inventory-with-langchain-openai-and-duckdb-f747245eaf4b)

In [12]:
file_path = "/Users/vamsi_mbmax/Library/CloudStorage/OneDrive-Personal/01_vam_PROJECTS/LEARNING/proj_Databases/dev_proj_Databases/pract-duckDB/data/inventory.csv"

- Use python approach of duckdb for below steps [loading data into mother duck]([https://](https://motherduck.com/docs/getting-started/connect-query-from-python/loading-data-into-md))
- the below approach is by using terminal

```yaml
# create database with duckdb
duckdb ./cdn_open_data.db

# duckdb create table from downloaded csv
D CREATE TABLE inventory as SELECT * from read_csv_auto('./inventory.csv');

# santiy check 
D show tables;
```

## CONNECT TO DATABASE

In [4]:
!pwd

/Users/vamsi_mbmax/Library/CloudStorage/OneDrive-Personal/01_vam_PROJECTS/LEARNING/proj_Databases/dev_proj_Databases/pract-duckDB/notebooks


In [5]:
# NOTE: dont forget to install `duckdb-engine` python package along w/ duckdb in requirements.txt
uri = "duckdb:////Users/vamsi_mbmax/Library/CloudStorage/OneDrive-Personal/01_vam_PROJECTS/LEARNING/proj_Databases/dev_proj_Databases/pract-duckDB/data/cdn_open_data.db"
connect_args = {"read_only": True}
CONN = create_engine(uri) # CONN -> engine_duckdb
CONN

Engine(duckdb:////Users/vamsi_mbmax/Library/CloudStorage/OneDrive-Personal/01_vam_PROJECTS/LEARNING/proj_Databases/dev_proj_Databases/pract-duckDB/data/cdn_open_data.db)

In [7]:
# load database
db = SQLDatabase.from_uri(
    database_uri=uri,
    include_tables=["inventory"],
    sample_rows_in_table_info=3,
)

db

/Users/vamsi_mbmax/Library/CloudStorage/OneDrive-Personal/01_vam_PROJECTS/LEARNING/proj_Databases/dev_proj_Databases/pract-duckDB/.venv/lib/python3.10/site-packages/duckdb_engine/__init__.py:173: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(


## LLM

In [45]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_community.utilities import SQLDatabase
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType

from langchain_openai import ChatOpenAI

# from langchain.llms import OpenAI
from langchain import HuggingFaceHub
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains import LLMChain

In [44]:
llm_langchain = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    temperature=0,
    model="gpt-3.5-turbo",
)

## DEFINE PROMPT

In [ ]:
template = """/
You are a SQL Analyst that is querying a database of Canada Open Data Inventory that about all the Canada Open Data from the Government of Canada website.

Below is a description of the columns, data types, and information in the columns:

The column name ref_number with the data type VARCHAR contains the following information: Unique identifier for every open data
The column name title_en with the data type VARCHAR contains the following information: English title of the open data
The column name title_fr with the data type VARCHAR contains the following information: French title of the open data
The column name description_en with the data type VARCHAR contains the following information: English description of the open data
The column name description_fr with the data type VARCHAR contains the following information: French description of the open data
The column name publisher_en with the data type VARCHAR contains the following information: Publisher name in English
The column name publisher_fr with the data type VARCHAR contains the following information: Publisher name in French
The column name date_published with the data type VARCHAR contains the following information: The date of this open data published
The column name language with the data type VARCHAR contains the following information: What language this open data
The column name size with the data type BIGINT contains the following information: The open data size
The column name program_alignment_architecture_en with the data type VARCHAR contains the following information: English name of the program alignment architecture
The column name program_alignment_architecture_fr with the data type VARCHAR contains the following information: French name of the program alignment architecture
The column name date_released with the data type VARCHAR contains the following information: The date this open data released
The column name portal_url_en with the data type VARCHAR contains the following information: English portal url
The column name portal_url_fr with the data type VARCHAR contains the following information: French portal url
The column name user_votes with the data type BIGINT contains the following information: The users votes count for this open data, which showing which open data are most request by the user
The column name owner_org with the data type VARCHAR contains the following information: Which org divison are the owner of this open data
The column name owner_org_title with the data type VARCHAR contains the following information: The org division title of the owner of this open data

Your job is to write and execute a query that answers the following question:
{query}
"""

# prompt = PromptTemplate.from_template(template)
prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['query'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='/\nYou are a SQL Analyst that is querying a database of Canada Open Data Inventory that about all the Canada Open Data from the Government of Canada website.\n\nBelow is a description of the columns, data types, and information in the columns:\n\nThe column name ref_number with the data type VARCHAR contains the following information: Unique identifier for every open data\nThe column name title_en with the data type VARCHAR contains the following information: English title of the open data\nThe column name title_fr with the data type VARCHAR contains the following information: French title of the open data\nThe column name description_en with the data type VARCHAR contains the following information: English description of the open data\nThe column name description_fr with the data type VARCHAR contains the following information: French descriptio

## DEFINE TOOL, AGENT

[langchain sqldatabase agent](https://python.langchain.com/v0.1/docs/integrations/toolkits/sql_database/)

In [39]:
# use the below if need to use w/ agents
tool_sql = SQLDatabaseToolkit(db=db, llm=llm_langchain)  # can be used w/ custom tools

In [51]:
from langchain_community.agent_toolkits import create_sql_agent

agent_sql = create_sql_agent(
    llm=llm_langchain,
    db=db,
    agent_type="openai-tools",
    # verbose=True,
)

## START QUERY

In [ ]:
agent_sql.invoke("What's the total open data record?")

In [ ]:
agent_sql.invoke("What's the most released org division title?")

In [52]:
agent_sql.invoke("what is the most voted open date between 2000 and 2023?")

{'input': 'what is the most voted open date between 2000 and 2023?',
 'output': 'The most voted open date between 2000 and 2023 is March 7, 2006, with 381 user votes.'}

In [53]:
agent_sql.invoke(
    "can you show me the top 10 open date in french title which have been published but never been released?"
)

{'input': 'can you show me the top 10 open date in french title which have been published but never been released?',
 'output': "The top 10 French titles that have been published but never been released are as follows:\n\n1. Requêtes d'informations financières et budgétaires des missions\n2. Adhérence au standard de service de livraison pour les requêtes de services en ligne pour les missions (SLM)\n3. Niveau de satisfaction au service en ligne pour les missions\n4. Liste des véhicules des services communs\n5. Rendu de décisions pour le comité sur la représentation à l'étranger (CORA)\n6. Biens des missions et demandes de matériel\n7. Requêtes de transport aux missions\n8. Mosaïque du Modèle numérique d'élévation de haute résolution (Mosaïque MNEHR) - Série CanÉlévation\n9. Le Système national de surveillance du déboisement (SNSD)\n10. Régions hydrogéologiques du Canada"}